# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
load_csv_file = "cities.csv"

weather_df = pd.read_csv(load_csv_file)

weather_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,beloha,-25.1667,45.0500,74.89,84.0,3.0,15.46,MG,1.645120e+09
1,1,rikitea,-23.1203,-134.9692,77.92,71.0,10.0,8.90,PF,1.645120e+09
2,2,kapaa,22.0752,-159.3190,67.12,69.0,75.0,4.61,US,1.645120e+09
3,3,dikson,73.5069,80.5464,-23.69,97.0,56.0,5.46,RU,1.645120e+09
4,4,jackson,42.4165,-122.8345,47.44,100.0,100.0,0.00,US,1.645120e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#Load g_key
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"]

# Transfer Lat and Lng numerical values into locations. 
locations = weather_df[["Lat", "Lng"]]

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Criteria for "Ideal Location" 70<temp<80, wind<10, cloudiness==0
idealweather_df = weather_df

#Filter for temp range, wind speed and cloudiness
idealweather_df = idealweather_df.loc[(idealweather_df["Max Temp"] < 80) & (idealweather_df["Max Temp"] > 70)]
idealweather_df = idealweather_df.loc[idealweather_df["Wind Speed"] < 10]
idealweather_df = idealweather_df.loc[idealweather_df["Cloudiness"] == 0]

# Drop any rows with null values
idealweather_df = idealweather_df.dropna()

idealweather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
167,167,east london,-33.0153,27.9116,70.74,78.0,0.0,3.44,ZA,1.645120e+09
209,209,pisco,-13.7000,-76.2167,73.45,68.0,0.0,8.05,PE,1.645121e+09
293,293,alibag,18.6411,72.8792,75.27,55.0,0.0,2.46,IN,1.645121e+09
352,352,sharjah,25.3573,55.4033,73.26,58.0,0.0,8.05,AE,1.645121e+09
412,412,olavarria,-36.8927,-60.3225,73.02,38.0,0.0,7.92,AR,1.645121e+09
414,414,alice springs,-23.7000,133.8833,71.15,68.0,0.0,4.61,AU,1.645121e+09
472,472,ixtapa,20.7000,-105.2000,78.03,53.0,0.0,6.91,MX,1.645121e+09
499,499,salalah,17.0151,54.0924,73.49,64.0,0.0,2.30,OM,1.645121e+09
503,503,hervey bay,-25.2986,152.8535,71.02,91.0,0.0,5.01,AU,1.645121e+09
581,581,byron bay,-28.6500,153.6167,71.55,83.0,0.0,5.66,AU,1.645121e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = idealweather_df

In [10]:
#set up parameters for the hotel search: within 5000 meters, type = lodging (for hotels), key = g_key
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [13]:
for index, row in hotel_df.iterrows():
    #pull latitudes and longitude from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # iterate over the locations
    params["location"] = f"{lat},{lng}"
    
    target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Now retrieving the query results for index {index}: {row['City']}.")
    response = requests.get(target_url, params=params).json()
    
    # extract results from query
    results = response['results']
    
    try:
        print(f"The closest hotel from your search is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No hotel found within your parameters, skipping")
        
    print("Search complete")

Now retrieving the query results for index 167: east london.
The closest hotel from your search is Tu Casa.
Search complete
Now retrieving the query results for index 209: pisco.
The closest hotel from your search is Hotel San Isidro Oficial.
Search complete
Now retrieving the query results for index 293: alibag.
The closest hotel from your search is Radisson Blu Resort & Spa - Alibaug, India.
Search complete
Now retrieving the query results for index 352: sharjah.
The closest hotel from your search is Swiss Belhotel Sharjah.
Search complete
Now retrieving the query results for index 412: olavarria.
The closest hotel from your search is Hotel Santa Rosa.
Search complete
Now retrieving the query results for index 414: alice springs.
The closest hotel from your search is Desert Palms Alice Springs.
Search complete
Now retrieving the query results for index 472: ixtapa.
The closest hotel from your search is hotel la terminal.
Search complete
Now retrieving the query results for index 499:

In [14]:
#Check df to make sure it captured all of the hotels
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
167,167,east london,-33.0153,27.9116,70.74,78.0,0.0,3.44,ZA,1.645120e+09,Tu Casa
209,209,pisco,-13.7000,-76.2167,73.45,68.0,0.0,8.05,PE,1.645121e+09,Hotel San Isidro Oficial
293,293,alibag,18.6411,72.8792,75.27,55.0,0.0,2.46,IN,1.645121e+09,"Radisson Blu Resort & Spa - Alibaug, India"
352,352,sharjah,25.3573,55.4033,73.26,58.0,0.0,8.05,AE,1.645121e+09,Swiss Belhotel Sharjah
412,412,olavarria,-36.8927,-60.3225,73.02,38.0,0.0,7.92,AR,1.645121e+09,Hotel Santa Rosa
414,414,alice springs,-23.7000,133.8833,71.15,68.0,0.0,4.61,AU,1.645121e+09,Desert Palms Alice Springs
472,472,ixtapa,20.7000,-105.2000,78.03,53.0,0.0,6.91,MX,1.645121e+09,hotel la terminal
499,499,salalah,17.0151,54.0924,73.49,64.0,0.0,2.30,OM,1.645121e+09,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
503,503,hervey bay,-25.2986,152.8535,71.02,91.0,0.0,5.01,AU,1.645121e+09,Shelly Bay Resort
581,581,byron bay,-28.6500,153.6167,71.55,83.0,0.0,5.66,AU,1.645121e+09,Backpackers Inn


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Create a layer for the hotel symbols
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


In [19]:
# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))